# <center>A Demonstration of K-Means Clustering
### <center>Applying this simple ML algorithm to image compression

K-means clustering is one of the simplest unsupervised machine learning algorithms used to identify clusters in quantitative datasets. The goal is to identify the the *K* clusters (where *K* is a number specified by the user) of *n* observations such that the Euclidian distance between observations in each cluster is minimised. This is a pretty old school algorithm with the idea going back to the 1950's. 

The basic idea is that the center positions of each of the *K* clusters, aka their "centroids", are initialized randomly in the multidimensional observation space. Next the distance of each datapoint to each of the *K* centroids is computed, and the datapoints are all assigned to the cluster associated with their nearest centroid. For each centroid, the mean position of all points assigned to that centroid is calculated, and the centroid is then shifted to this mean position. A simple 2D illustration of this process is shown below:

<img src="test.gif" width=800 />

Here the centroid positions are shown by colored stars and at each iteration points assigned to each centroid are colored to match their centroid. *(The animation is created with Kmeans_2d_animation.py, included in the repo)*

## <center>K-Means Clustering for Image Compression

<img src="RGB_color_solid_cube.png" width=500 />

*image credit: wikipedia user Datumizer*

The K-means clustering algorithm is not limited to 2D data as in the example above however, but can be generalised to higher dimensions as well. One useful application is in image compression. Digital color images can be representated as a 3D matrix consisting of 3 2D channels corresponding to a red, green, and blue image (or 4 channels in some cases, e.g. PNG, with the fourth representing the "alpha" or transparency value). In RGB24 the value in each channel for any given pixel can take a value between 0 and 255 resulting in 256 cubed, or 16,777,216 possible colors. The term 24RGB comes from the fact that 8 bits are required to specify which of the 256 values applies to each pixel, thus 8x3=24. You can imagine that for even relatively small images storing 24 bits per pixel can add up pretty quickly. For instance, the iphone 11 has a 12 megapixel camera, so storing 24 bits (3 bytes) per pixel would result in 36 Mb per image.

Most images, however, do not use all 16+ million colors avaialable so a way to reduce to storage size is to use indexed coloring. This is a method where all of the possible colors that occur in a given image are assigned a unique index and the image is stored as a single value for each pixel corresponding to the index assigned to the color of that pixel. The result of indexed coloring is that the image file size is directly correlated with the number of colors that appear in the image. This means that, for a given image, if you can reliably represent the image using fewer colors, the file size of that image can be significantly reduced.

This is where K-means clustering comes in. Remember that each pixel can be represented by three numerical values, the R, G, and B channels. In this way we can think of each pixel as an individual datapoint in a 3D numerical space. In this context we can apply K-means clustering by initializing *K* centroids at random RGB colors and apply the pixel assigning and centroid moving as in the 2D example above. In this case, we can specify how many colors the final image will contain with the value of *K*. The lower a value of *K* we choose, the smaller the final image size will be (though this compression will come at the cost of image quality). Ok, now for the code:

In [9]:
import numpy as np
import imageio
import matplotlib.pyplot as plt
from PIL import Image

class Image_Compressor():
    
    def __init__(self,image_array,K=16):
        # Here the initial values of the centroids are created. In this case I'm making
        # half of them close to black and half close to white, but this is only for
        # illustrative purposes as it slows down the convergence of the algorithm to
        # create more interesting animations of the process. In general, simply initiating
        # to random values between 0 and 255 is fine (and faster!).
        vals = np.concatenate((np.random.uniform(0,1,int(K/2)),np.random.uniform(254,255,int(K/2))))
        self.centroids = np.copy(vals)
        for i in range(2): self.centroids=np.vstack((self.centroids,np.copy(vals)))
        self.centroids = self.centroids.T
        
        # In general it's probably worthwhile to also include a loss here and measure the
        # difference between the input and output images. You can then define a minimum 
        # change in the loss between two iterations at which to stop iterating and output
        # the final image. This code is mostly just to illustrate the technique so I've
        # left that out.
        
        # Reshape the image into an array with dimensions (npixels, 3)
        self.X  = np.reshape(image_array,(image_array.shape[0]*image_array.shape[1],3))
        # array C will store the centroids each pixel is assigned to
        self.C  = np.zeros(self.X.shape[0])
        self.K  = K
        self.r  = 0
        # Save the original image dimensions for reconstructing the ouput
        self.xin,self.yin = image_array.shape[0],image_array.shape[1]
        
    # Assign each pixel to the nearest color
    def assign_points(self):
        for i in range(len(self.C)):
            self.r=np.sqrt(np.sum((self.centroids-self.X[i])*(self.centroids-self.X[i]),axis=1))
            self.C[i] = np.where(self.r == np.min(self.r))[0][0]
            
    # Shift the colors towards the mean of matched pixels. Here I've included a
    # "learning rate", which slows down how far the centroids are moved each time.
    # This is not actually necessary, but allows me to slow down the algorithm further,
    # again for illustrative purposes.
    def move_centroids(self,lrate=1.):
        for i in range(self.K):
            t = np.where(self.C == i)[0]
            if len(t) > 0:
                true_cent = np.array([np.mean(self.X[:,0][t]),np.mean(self.X[:,1][t]),np.mean(self.X[:,2][t])])
                diff = self.centroids[i].astype(float)-true_cent
                self.centroids[i] -= (diff*lrate)
            else:
                # poor poor unused colors :'(
                pass

    # Reconstruct the current image. Remember that the input was reshaped
    # to have dimensions (npixels,3)
    def remake_im(self,output_name):
        imout = np.zeros(self.X.shape).astype(np.uint8)
        for i in range(self.X.shape[0]):
            imout[i] = self.centroids[self.C[i].astype(np.uint8)]

        imout = imout.reshape((self.xin,self.yin,3))
        self.im = imout
        imout = Image.fromarray(imout)
        imout.save(f'{output_name}')
        
    def iterate(self,lrate=1.,output=False,output_name='frame.jpg'):
        self.assign_points()
        self.move_centroids(lrate=lrate)
        if output: self.remake_im(output_name)
    

The above code will take the image you want to compress as input and create an Image_Compressor object with the key function for running the algorithm being "iterate". This will go through one iteration of assigning points, moving the centroids, and optionally outputting the image. When the Image_Compressor object is created, you can also specify the number of colors in the output image using the kwarg, "K". The default is to create a 16 color image. The image we'll be working with is this image of a selection of marbles:

<img src="marb.jpg" width=600 />

In the code below, we'll also output the image after every iteration and put them all together in an animation. Things to note here are that I've used a "learning rate" parameter to slow down the moving of the centroids and I've initialised all the centroids to be either nearly black ro nearly white. These simply slow everything down for the purposes of the animation and serve no other real purpose. For the animation we'll use K=32, 32 colors in the final image.

In [10]:
# first load the image.
input_image = np.asarray(imageio.imread('marb.jpg'))

compressor  = Image_Compressor(input_image,K=32)

# Initialise some stuff. I'm using a variable learning rate and also creating
# a list to save all the images in that will be used to create the animation at the end
images = []
xx = np.arange(0,100,1)
lrates = np.exp(xx**(0.4))
lrates = (lrates/np.max(lrates))*0.9
lrates += 0.1

# Now run the iterations. The image is saved/overwritten each time
# and stored in an array.
for i in range(len(lrates)+25):
        # Catch because len(lrates) < n_iterations
        try:
            compressor.iterate(lrate=lrates[i],output=True)
        except:
            compressor.iterate(output=True)
            
        images.append(imageio.imread('frame.jpg'))
        
imageio.mimsave('compress.gif', images)

Here's the resulting animation illustrating the compression process:

<img src='compress.gif' width=500>

For comparison we'll also create compressed versions using fewer colors. We'll create versions containing 2, 8, and 32 colors.

In [21]:
comp8 = Image_Compressor(input_image,K=2)
comp16 = Image_Compressor(input_image,K=8)
comp32 = Image_Compressor(input_image,K=32)

# Now iterate on all three. This time we'll just use lrate=1.0 so it won't
# take too many iterations to converge:
for i in range(50):
    for comp in [comp8,comp16,comp32]: comp.iterate()

# Now output the final images
comp8.remake_im('comp2.jpg')
comp16.remake_im('comp8.jpg')
comp32.remake_im('comp32.jpg')

<img src='comp2.jpg' width=400> <img src='comp8.jpg' width=400> <img src='comp32.jpg' width=400>

From top to bottom these show the 2, 8, and 32 color versions of the input. You can see that the image quality increases towards to bottom as expected. Finally we can check the level of compression each time:

In [2]:
import os

# Get the size of each image in kb
input_kb = os.stat('marb.jpg').st_size/1.e3
comp8_kb = os.stat('comp2.jpg').st_size/1.e3
comp16_kb = os.stat('comp8.jpg').st_size/1.e3
comp32_kb = os.stat('comp32.jpg').st_size/1.e3

print(f'Input: {input_kb} kb')
print(f'2 Color: {comp8_kb} kb')
print(f'8 Color: {comp16_kb} kb')
print(f'32 Color: {comp32_kb} kb')

Input: 77.036 kb
2 Color: 29.578 kb
8 Color: 40.216 kb
32 Color: 38.523 kb


We see that the 2 color image has the highest level of compression while the 8 and 32 are fairly similar at around 50% of the original image size. It should be noted, however, that here we are using JPEG images which is itself a highly compressed ("lossy" compression) format. The original image used here was actually a PNG with a file size of 306 kb, thus compressing to fewer colors and converting to JPEG results in a compression to roughly 13% of the original size. 